#  Setup
Run below cells and hide it afterwards with the arrow on the left. 

In [0]:
!pip install gym[Box2D] pyvirtualdisplay pyglet > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1

In [0]:
import gym
from gym import logger as gymlogger
from gym.wrappers import Monitor
gymlogger.set_level(40) #error only
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (10.0, 8.0)
import math
import glob
import io
import base64
from IPython.display import HTML

from typing import List, Tuple

import torch
from torch import nn
import torch.nn.functional as F
from collections import deque

from IPython import display as ipythondisplay
from IPython.display import display, update_display, clear_output
from time import sleep

from pyvirtualdisplay import Display
xdisplay = Display(visible=0, size=(1300, 900), backend="xvfb")
xdisplay.start()


"""
Utility functions to enable video recording of gym environment and displaying it
To enable video, just do "env = wrap_env(env)""
"""

class DoneWrapper(gym.Wrapper):

  def step(self, action):
    observation, reward, done, info = self.env.step(action) 
    return observation, reward, False, info
      

def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    
    
def wrap_env(env, done=True):
  if not done:
    env = DoneWrapper(env)
  env = Monitor(env, './video', force=True, mode='evaluation')
  return env


def print_ansi(screen, display_id='42', wait=0.5):
    clear_output(wait=True)
    update_display(print(screen.getvalue()), display_id=display_id)
    sleep(wait)


def plot(img):
  fig = plt.figure(figsize=(8,6))
  ax = fig.add_subplot(111)
  ax.imshow(img)
  ax.set_xticks([])
  ax.set_yticks([])

In [0]:
def gather_trajectories(env: gym.Env, policy, num_trajs: int = 10):
    """Gather `num_trajs` trajectories by interacting with the environment using the given policy."""
    
    # preapre a list for the trajectories
    history = []
    
    for traj_idx in range(num_trajs):
        obs = env.reset()
        done = False
        current_traj = []
        while not done:
            
            # sample an action from the policy
            action = policy.sample(obs) 
            # feed it into the environment
            next_obs, reward, done, _ = env.step(action)
            
            # save into the history
            current_traj += [(obs, action, reward)] 

            obs = next_obs
        history += [current_traj]
        
    return history


def calculate_return(rewards: List[float]) -> Tuple[float, List[float]]:
    """Calulated and episode and step returns"""
    # calculate the sum of rewards from the episode
    rewards = np.array(rewards)
    episode_return = np.sum(rewards)
    
    # prepare a list for the step returns
    step_returns = []

    # calculate discounted return for each step
    # hint: it's easier to go backwards
    step_returns = [rewards[-1]]
    for reward in reversed(rewards[:-1]):
     
        last_return = step_returns[-1]
        step_returns += [reward + last_return]
    step_returns.reverse()

    return episode_return, step_returns


def process_trajectories(history: List):
    """Process gathered trajectories into tensors and calculate returns"""
    # prepare containers for each element
    obs_array = []
    action_array = []
    return_array = []
    episode_returns = []
    
    # loop over the whole history
    rewards = []
    for traj_idx, traj in enumerate(history):
        # unpack the elements
        traj_obs, traj_actions, traj_rewards = list(zip(*traj))

        # process the end of an episode 
        # - calculate episode and step returns
        # ???
        episode_return, step_returns = calculate_return(traj_rewards)
        
        episode_returns += [episode_return]
        obs_array += traj_obs
        action_array += traj_actions
        return_array += step_returns

    # cast out data to tensors (will be useful later)     
    obs_array = torch.tensor(obs_array, dtype=torch.float32)
    action_array = torch.tensor(action_array, dtype=torch.float32)
    return_array = torch.tensor(return_array, dtype=torch.float32)
    episode_returns = torch.tensor(episode_returns, dtype=torch.float32)
    
    return obs_array, action_array, return_array, episode_returns


def visualize(env, policy):
    """Run the provided policy on the environment"""

    env = wrap_env(env)
    obs = env.reset()
    done = False
    
    while not done:
        action = policy.sample(obs)
        obs, reward, done, _ = env.step(action)
        env.render()

    env.close()
    show_video()


class NetworkPolicy(nn.Module):

    def __init__(self, obs_dim: int, action_dim: int, h_dim: int = 16):
        super(NetworkPolicy, self).__init__()

        self.model = nn.Sequential(nn.Linear(obs_dim, h_dim),
                                   nn.Tanh(),
                                   nn.Linear(h_dim, action_dim))

    def probs(self, obs):
        # cast the numpy array to a torch tensor if necessary
        if not isinstance(obs, torch.Tensor):
            obs = torch.tensor(obs, dtype=torch.float32)
        # get logits from the model
        logits = self.model(obs) 
        # use softmax function to transform logits into probability distribution
        return F.softmax(logits, -1) 

    def log_probs(self, obs: np.ndarray):
        # cast the numpy array to a torch tensor if necessary
        if not isinstance(obs, torch.Tensor):
            obs = torch.tensor(obs, dtype=torch.float32)
        # get logits from the model
        logits = self.model(obs) 
        # use *log* softmax function to transform logits into probability distribution
        return F.log_softmax(logits, -1) 
        
    def sample(self, obs):
        # again, sample from the prepared probability vector 
        # remember the `.item()` method!
        probs = self.probs(obs)
        return torch.multinomial(probs, 1).item()


def policy_gradient_step(policy: NetworkPolicy,
                         optimizer: torch.optim.Optimizer, 
                         obs: torch.Tensor, 
                         actions: torch.Tensor, 
                         step_returns: torch.Tensor,
                         num_trajs: int):

    # pass the obs to the policy to get log probabilities of each action
    log_probs = policy.log_probs(obs)
    
    # get the probability of the action thast was actual performed for each observation
    actions = actions.view(-1, 1).long()
    action_log_probs = log_probs.gather(1, actions).squeeze()
    # calculat the gradient
    target = -(action_log_probs * step_returns).sum() / num_trajs
    # pass it to the optimizer
    optimizer.zero_grad()
    target.backward()
    optimizer.step()


def get_value_network(env: gym.Env, h_dim: int = 32):
    """Create a value netowrk with 2 hidden layers, both with `h_dim` neurons
       and Tanh nonlinear activations"""

    obs_dim = env.observation_space.shape[0]
    
    # build the network
    value_network = nn.Sequential(nn.Linear(obs_dim, h_dim),
                                  nn.Tanh(),
                                  nn.Linear(h_dim, h_dim),
                                  nn.Tanh(),
                                  nn.Linear(h_dim, 1))
                            
    return value_network


def value_net_step(obs: torch.Tensor, 
                   step_returns: torch.Tensor,
                   model: torch.nn.Module, 
                   optim: torch.optim.Optimizer):
    """"Train the value network on a single batch of states and returns"""
    
    # pass the observatrion to get network and get the predicted values
    values = model(obs).squeeze()

    # calculate the loss function
    loss = ((values - step_returns) ** 2).mean()
    
    # pass gradeints to the optimizer
    optim.zero_grad()
    loss.backward()
    optim.step()

# Part 4. Proximal Policy Optimization

In this part we'll finally implement the PPO algorithm, similarly to previous steps we'll start with a single algorithm step to then use it in a training function.

## Exercise: PPO Step

Implement the following PPO step. As a reminder here the formula for PPO update:

$$ \mathcal{L}(\theta) = \mathbb{E}_t \Big[ \min \big(\rho A_t, \text{clip}(\rho, 1 - \epsilon, 1 + \epsilon) A_t\big) \Big] $$

$$ \rho = \frac{\pi_\theta (a_t | s_t)}{\pi_{\theta OLD}(a_t | s_t)} $$

We approximate the expected value by sampling and in order to maximize this function we need to pass a negative of this function to the PyTorch optimizer, so the loss function that we in fact implement has the form of:
$$
-\frac{1}{T} \sum_i^T \Big[ \min \big(\rho A_t, \text{clip}(\rho, 1 - \epsilon, 1 + \epsilon) A_t\big) \Big]
$$

In [0]:
PPO_EPS = 0.2 

def ppo_step(policy: torch.nn.Module, 
             optimizer: torch.optim.Optimizer, 
             obs: torch.Tensor, 
             actions: torch.Tensor, 
             advantages: torch.Tensor, 
             old_probs: torch.Tensor):
    
    # get action probs for each possible action from the policy
    probs = ???
    
    # we need to detach the old probs as the updates need to be based on current state only
    old_probs.detach()

    # gather the probability of the actual action that was taken for each state
    action_probs =  ???
    
    # do the same for old probabilities
    old_action_probs = ???
    
    # callulate rho, the ratio of the current to old probabilities
    prob_ratio = ???
    
    # calculate the unclipped objective, i.e. ratio times advantage
    unclipped_objective = ???
    # calculate the clipped objective, i.e. clipped ratio times advantage
    # hint: torch.clamp
    clipped_objective = ???
    # calculate the min of the two objectives
    # hint: torch.stack
    stacked_objectives = ???
    objective = ???
    # the nial target should be the minus average of the objective
    target = ???
    
    # with PPO target ready we can now pass it to our optimizer
    optimizer.zero_grad()
    target.backward()
    optimizer.step()

Now that we have our PPO step function we can use it in training, implement the `train_ppo` procedure following the instructions in the comments.

In [0]:
def train_ppo(env, policy, value_network, num_iterations=100, batch_size=64, trajs_per_gather=10, num_miniepochs=3):
    
    # prepare optimizers for both networks
    optimizer = torch.optim.Adam(policy.parameters(), lr=5e-3)
    value_optimizer = torch.optim.Adam(value_network.parameters(), lr=5e-1)
    
    # training loop
    for idx in range(num_iterations + 1):

        # gather trajectories using current policy
        history = gather_trajectories(env, policy, trajs_per_gather)
        # calculate the obs, actions and returns array by processing the trajectories
        obs, actions, rets, ep_returns = process_trajectories(history)
        
        # get values from the value netowork and calculate the advantage
        # don't forget to detach the value network's output from the graph!
        values = ???
        advs = ???

        # randomize the order of trajectory steps for PPO!
        indices = torch.randperm(len(obs))
        # calculate the required number of batches
        batch_num = len(obs) // batch_size

        # prepare old probabilities vector 
        old_probs = policy.probs(obs).detach()
        
        # ppo traning miniloop
        for miniepoch_idx in range(num_miniepochs):
            # PPO batches loop
            for batch_idx in range(batch_num):

                # gather the samples for this batch
                batch_start = batch_size * batch_idx
                batch_end = batch_start + batch_size
                batch_indices = indices[batch_start:batch_end]

                # run ppo training step
                ppo_step(policy=policy,
                         optimizer=optimizer,
                         obs=obs[batch_indices],
                         actions=actions[batch_indices],
                         advantages=advs[batch_indices],
                         old_probs=old_probs[batch_indices])
                
                # run the value network traning step
                value_net_step(obs=obs[batch_indices], 
                               step_returns=rets[batch_indices],
                               model=value_network, 
                               optim=value_optimizer)
                
        if idx % 10 == 0:
            print(f"Traning iteration {idx}, mean episode returns: {ep_returns.mean():.3f}")

All that's left is to run the PPO algorithm on our environment.

In [0]:
# moon lander
env = gym.make("LunarLander-v2")
# or cart pole
# env = gym.make("CartPole-v1")

# gather necessary dimensions for our netowrk
obs_dim = env.observation_space.shape[0]
action_dim = env.action_space.n

# initialize the policy
network_policy = NetworkPolicy(obs_dim, action_dim)
value_network = get_value_network(env)

# train the model
train_ppo(env, 
          policy=network_policy, 
          value_network=value_network,
          num_iterations=50,
          trajs_per_gather=20)

In [0]:
visualize(env, network_policy)

## Bonus Exercise 1: Compare PPO with vanilla and baseline PG
* Compare the required number of samples to reach some desired episode return.
* Compare them on the episode return after the same, fixed number of samples seen.  

## Bonus Exercise 2: Sampling from a very different agent
Sample trajectories using the random policy defined in the first part of this workshop. Try to use them as the "old policy" in PPO and train it this way. Does it work? Why or why not? 

## Uber Bonus Exercise: Different Solutions to Trust Region
Can you come up with a different way to force $\pi_\theta$ to stay fairly close to its previous values in distribution space than the clipping used in PPO. Experiment with your ideas, see how they perform.
